# Compound Subgraphs
https://github.com/graphprotocol/compound-V2-subgraph

# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path

path = Path('/content/drive/MyDrive/colab_notebooks/algovera/defi')

In [3]:
import json

with open(path/"DataGathering/creds.json") as f:


    creds = json.loads(f.read())["ethkey"]

# API Calls

In [4]:
import json
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# Turn timestamp into block number
def get_block(s: requests.Session, timestamp: int) -> str:
    ETHERSCAN_KEY = ''
    ETHERSCAN_KEY_API_URL = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey={creds}'

    block_response = s.request('GET', ETHERSCAN_KEY_API_URL)
    block = block_response.json()['result']

    return block

#https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2

def query_compound_subgraph(dt):

    compoundV2_api_url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2'

    retries = Retry(total=5, backoff_factor=2, status_forcelist=[429, 502, 503, 504])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}


    block = get_block(s, int(dt))
    body_Compound = {
        "query": "{ markets (first: 100,block: {number: %s})  { id borrowRate cash collateralFactor exchangeRate interestRateModelAddress name"\
                    " reserves supplyRate  symbol totalBorrows totalSupply underlyingAddress underlyingName underlyingPrice underlyingSymbol reserveFactor  underlyingPriceUSD}}" % block}
    
    
    
    compound_json_file = json.dumps(body_Compound)
    response = s.request('POST', compoundV2_api_url, headers=headers, data=compound_json_file)

    return response


# Time Range

In [43]:
t = int(datetime.timestamp(datetime.now()));t

1655885321

In [18]:
date = datetime.fromtimestamp(1652208200, tz=timezone.utc)
end_date = datetime.fromtimestamp(t, tz=timezone.utc)

dates = pd.date_range(date,end_date,freq="30T")

timestamps = [pd.Timestamp(d).timestamp() for d in dates]

In [19]:
len(timestamps)

2041

In [44]:
1652208200 - 1655885321

-3677121

# Main Loop

In [28]:
def fetch_data(timestamps):

    for dt in tqdm(timestamps):

        data = list()
        kickout = list()

        response = query_compound_subgraph(dt)

        try:

            for token in response.json()['data']['markets']:

                    data.append(token)

            df = pd.DataFrame(data)

            df["timestamp"] = dt
            
            df.to_csv(path/f"DataGathering/compoundV2/compound_mainnet_30_May-Current/{int(dt)}_compound.csv",index=False)

        except KeyError:
            kickout.append(dt)
    
    return kickout


In [41]:
kickout = [True]
i = 0
while kickout:

    if not kickout:
        kickout = fetch_data(kickout)
    else:
        kickout = fetch_data(timestamps)

    print(len(kickout))
    i+=1

100%|██████████| 2041/2041 [13:15<00:00,  2.56it/s]

0


In [36]:

if not kickout:
    kickout = fetch_data(timestamps)
else:
    kickout = fetch_data(kickout)

100%|██████████| 1/1 [00:00<00:00,  2.84it/s]


In [ ]:
df.query('''
underlyingAddress == "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48" | \
underlyingAddress == "0x0000000000000000000000000000000000000000" | underlyingAddress == "0x6b175474e89094c44da98b954eedeac495271d0f" | \
underlyingAddress == "0xdac17f958d2ee523a2206206994597c13d831ec7"
                        '''
                        ).reset_index(drop=True).sort_values("timestamp",ascending=True)[["underlyingSymbol","totalBorrows","totalSupply","borrowRate","supplyRate","exchangeRate","underlyingPrice"]]

,underlyingSymbol,totalBorrows,totalSupply,borrowRate,supplyRate,exchangeRate,underlyingPrice
0,USDC,426079949.147381,47703642858.55778568,0.0197582075505504,0.0072221551642656,0.022602788095407,0.000485
1,ETH,57572.93997566797470515,40124677.54166306,0.0271453656389184,0.0015530502636288,0.020063515115755748,1
2,DAI,488821681.632796978277185064,38183854787.55375299,0.029094926128992,0.0143907503493024,0.022000038277155387,0.000486317635483215
3,USDT,364904972.850082,25989755475.98693577,0.032148406025136,0.0191201201838144,0.0218367573383649,0.000489


In [ ]:
f"{(57572.93997566797470515 * 2068.25):,}"

'119,075,233.1046753'